In [ ]:
########################################
#getting system arguments
import sys
def GetArg_dataName(default="Variables"):
    """
    Safely retrieve dataName from sys.argv.
    #Run One: python Tracked_Profiles.py Variables
    #Run Two: python Tracked_Profiles.py Entrainment
    #Run Three: python Tracked_Profiles.py PROCESSED_Entrainment
    #Run Four: python Tracked_Profiles.py W_Budgets
    #Run Five: python Tracked_Profiles.py QV_Budgets
    #Run Six: python Tracked_Profiles.py TH_Budgets
    """
    # If run inside Jupyter, sys.argv will include ipykernel arguments
    if any("ipykernel_launcher" in arg for arg in sys.argv):
        print(f"Using default dataName: {default}")
        return default

    # If a user-specified argument exists, use it
    if len(sys.argv) > 1:
        out=sys.argv[1]
        print(f"Using argument dataName: {out}")
        return out

    return default

dataName = GetArg_dataName()

In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py
from tqdm import tqdm

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import *

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracking_Algorithms", dataName="Lagrangian_UpdraftTracking",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#data manager class (for saving data)
DataManager_TrackedProfiles = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracked_Profiles", dataName="Tracked_Profiles",
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import TrackingAlgorithms_DataLoading_Class, Results_InputOutput_Class, TrackedParcel_Loading_Class

In [ ]:
# IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","3_Tracked_Profiles"))
from CLASSES_TrackedProfiles import TrackedProfiles_DataLoading_CLASS

In [ ]:
##############################################
#JOB ARRAY

In [ ]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res,t_res):
    if res=='1km':
        if t_res=='5min':
            num_jobs=20
        elif t_res=='1min':
            num_jobs=100
    elif res=='250m': 
        if t_res=='1min':
            num_jobs=500
    return num_jobs
num_jobs = GetNumJobs(ModelData.res,ModelData.t_res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    loop_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return loop_elements
loop_elements = GetNumElements()

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
def MakeDataDictionary(variableNames,t,printstatement=False):
    timeString = ModelData.timeStrings[t]
    # print(f"Getting data from {timeString}","\n")
    
    dataDictionary = {variableName: CallLagrangianArray(ModelData, DataManager, timeString, variableName=variableName, printstatement=printstatement) 
                      for variableName in variableNames}      
    return dataDictionary
    
def GetSpatialData(t):    
    variableNames = ['Z']
    dataDictionary = MakeDataDictionary(variableNames,t)
    [Z] = (dataDictionary[k] for k in variableNames)
    return Z

In [ ]:
####################################
#RUN SETUP

In [ ]:
#data variable list
def GetVarNames(dataName): 
    if dataName=="Variables":
        varNames = ['W', 'QV','QCQI', 'RH_vapor', 'THETA_v', 'THETA_e', 'MSE', 'HMC', 'CONVERGENCE','VMF_g','VMF_c']
    elif dataName == "Entrainment":
        varNames = ['D_c', 'D_g', 'E_c', 'E_g',
                          'TransferD_c', 'TransferD_g', 
                          'TransferE_c', 'TransferE_g']
    elif dataName == "PROCESSED_Entrainment":
        varNames = ['PROCESSED_D_c', 'PROCESSED_D_g', 
                    'PROCESSED_E_c', 'PROCESSED_E_g',
                    'PROCESSED_TransferD_c', 'PROCESSED_TransferD_g', 
                    'PROCESSED_TransferE_c', 'PROCESSED_TransferE_g']
    elif dataName=="W_Budgets":
        varNames = ['WB_BUOY', 'WB_HADV', 'WB_HIDIFF', 'WB_HTURB',
                    'WB_PGRAD', 'WB_VADV', 'WB_VIDIFF', 'WB_VTURB']
    elif dataName=="QV_Budgets":
        varNames = ['QVB_HADV', 'QVB_HIDIFF', 'QVB_HTURB', 'QVB_MP',
                    'QVB_VADV', 'QVB_VIDIFF', 'QVB_VTURB']
    elif dataName=="TH_Budgets":
        varNames = ['PTB_DISS', 'PTB_DIV', 'PTB_HADV', 'PTB_HIDIFF', 
                    'PTB_HTURB', 'PTB_MP', 'PTB_RAD', 'PTB_VADV', 
                    'PTB_VIDIFF', 'PTB_VTURB']
    return varNames

In [ ]:
########################################
#RUNNING FUNCTIONS

In [ ]:
#Functions for Initializing Profile Arrays
def CopyStructure(dictionary, placeholder=None):
    """Deep-copy dictionary structure, replacing leaves with a given placeholder."""
    if isinstance(dictionary, dict):
        return {k: CopyStructure(v, placeholder) for k, v in dictionary.items()}
    else:
        return placeholder

def InitializeProfileArrays(trackedArrays, varNames, zhs=ModelData.zh):
    """
    Create a new dictionary with the same nested structure as trackedArrays,
    and for each variable name, create:
        - 'profile_array' / 'profile_array_squares'
        - 'profile_array_left' / 'profile_array_left_squares'
        - 'profile_array_right' / 'profile_array_right_squares'
    Each array has shape (len(zhs), 3) and zhs in the last column.
    """
    profileArraysDictionary = {}

    for category, depth_dict in trackedArrays.items():  # e.g. 'CL', 'SBF'
        profileArraysDictionary[category] = {}

        for depth_type in depth_dict.keys():  # e.g. 'ALL', 'SHALLOW', 'DEEP'
            profileArraysDictionary[category][depth_type] = {}

            for varName in varNames:
                # Create base profile array
                base_profile = np.zeros((len(zhs), 3))
                base_profile[:, 2] = zhs

                profileArraysDictionary[category][depth_type][varName] = {
                    # Main / all parcels
                    "profile_array": base_profile.copy(),
                    "profile_array_squares": base_profile.copy(),

                    # Left subset (-1)
                    "profile_array_left": base_profile.copy(),
                    "profile_array_left_squares": base_profile.copy(),

                    # Right subset (+1)
                    "profile_array_right": base_profile.copy(),
                    "profile_array_right_squares": base_profile.copy(),
                }
    return profileArraysDictionary

In [ ]:
def GetParcelNumbers(trackedArray, t):
    """
    Return all parcel indices (p) and their corresponding row indices
    for parcels that are active at time t.
    Vectorized, no row-by-row loops.
    """
    t_start = trackedArray[:, 1]
    t_end   = np.minimum(trackedArray[:, 2] + trackedArray[:, 3], ModelData.Ntime)

    # Boolean mask for rows active at time t
    mask = (t >= t_start) & (t <= t_end)

    # Extract parcel numbers and their corresponding row indices
    selectedRows = np.where(mask)[0]
    selectedPs = trackedArray[selectedRows, 0]
    leftRightIndexes = trackedArray[selectedRows, 4]

    return selectedRows, selectedPs, leftRightIndexes

In [ ]:
def MakeTrackedProfiles(trackedArrays,profileArraysDictionary,varNames,VARs,Z,t):
    """
    Update profileArraysDictionary with variable data for parcels active at time t.
    Accumulates sums and counts in both profile_array and profile_array_squares.
    """
    #CALCULATING
    for key1, subdict in trackedArrays.items():         # e.g. 'CL', 'SBF'
        print("\t",f'working on {key1}')
        for key2, trackedArray in subdict.items():           # e.g. 'ALL', 'DEEP'
            print("\t\t",f'working on {key2}')
            #loading the profile array to fill
            profileArray = profileArraysDictionary[key1][key2] 
    
            #getting parcels in trackedArray to run through
            _, selectedPs, leftRightIndexes = GetParcelNumbers(trackedArray, t) #get parcels that are counted at time t
            
            #getting Z data
            zLevels = Z[selectedPs]

            # Boolean masks
            mask_left = leftRightIndexes == -1
            mask_right = leftRightIndexes == 1
            
            for varName in varNames:
                #getting data
                results = VARs[varName][selectedPs]
                
                 # --- MAIN appending (all parcels go here) ---
                np.add.at(profileArray[varName]["profile_array"][:, 0], zLevels, results)
                np.add.at(profileArray[varName]["profile_array"][:, 1], zLevels, 1)
                np.add.at(profileArray[varName]["profile_array_squares"][:, 0], zLevels, results**2)
                np.add.at(profileArray[varName]["profile_array_squares"][:, 1], zLevels, 1)
                
                # --- LEFT subset (-1 only) ---
                if np.any(mask_left):
                    np.add.at(profileArray[varName]["profile_array_left"][:, 0], zLevels[mask_left], results[mask_left])
                    np.add.at(profileArray[varName]["profile_array_left"][:, 1], zLevels[mask_left], 1)
                    np.add.at(profileArray[varName]["profile_array_left_squares"][:, 0], zLevels[mask_left], results[mask_left]**2)
                    np.add.at(profileArray[varName]["profile_array_left_squares"][:, 1], zLevels[mask_left], 1)

                # --- RIGHT subset (+1 only) ---
                if np.any(mask_right):
                    np.add.at(profileArray[varName]["profile_array_right"][:, 0], zLevels[mask_right], results[mask_right])
                    np.add.at(profileArray[varName]["profile_array_right"][:, 1], zLevels[mask_right], 1)
                    np.add.at(profileArray[varName]["profile_array_right_squares"][:, 0], zLevels[mask_right], results[mask_right]**2)
                    np.add.at(profileArray[varName]["profile_array_right_squares"][:, 1], zLevels[mask_right], 1)

    return profileArraysDictionary

In [ ]:
########################################
#RUNNING

In [ ]:
#Loading in Tracked Parcels Info
trackedArrays,LevelsDictionary = TrackedParcel_Loading_Class.LoadingSubsetParcelData(ModelData,DataManager,
                                                         Results_InputOutput_Class)

#Removing After Ascent Count for SHALLOW parcels
#Reason is there is a lot of shallow parcels
#that hit their peak below 4 km, 
#but stay in the cloud in turbulent eddies and later exit at much higher levels
#and exit the cloudy updraft higher
# for key1, subdict in trackedArrays.items():
#     subdict['SHALLOW'][:,3]=0
#testing without for now #*

# Get Variable Names
varNames = GetVarNames(dataName)

In [ ]:
for t in tqdm(loop_elements, desc="Processing"):
    if dataName in ['Entrainment','PROCESSED_Entrainment'] and t == ModelData.Ntime-1:
        continue
    print("#" * 40,"\n",f"Processing timestep {t}/{loop_elements[-1]}")
    timeString = ModelData.timeStrings[t]

    #Forming Dictionary for Profile Arrays for current timestep
    trackedProfileArrays = CopyStructure(trackedArrays)
    profileArraysDictionary = InitializeProfileArrays(trackedProfileArrays,varNames)
    
    #getting variable data
    VARs = MakeDataDictionary(varNames, t)
    Z = GetSpatialData(t)

    #making tracked profiles
    profileArraysDictionary = MakeTrackedProfiles(trackedArrays,profileArraysDictionary,varNames,VARs,Z,t)

    #saving tracked profiles for current timestep
    TrackedProfiles_DataLoading_CLASS.SaveProfile(ModelData,DataManager_TrackedProfiles, profileArraysDictionary, dataName, t)

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
# recombine=True

In [ ]:
import copy
def RecombineProfiles(ModelData, DataManager):
    """
    Combine tracked profiles across all timesteps using the first as a template.
    """
    print(f"Recombining {ModelData.Ntime} TrackedProfiles files...\n")

    trackedProfileArrays = None

    for t in tqdm(range(ModelData.Ntime), desc="Combining Profiles", unit="timestep"):
        if dataName in ['Entrainment','PROCESSED_Entrainment'] and t == ModelData.Ntime-1:
            continue
        profileArraysDictionary = TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData, DataManager, dataName, t)

        if t == 0:
            # Deep copy structure so we don’t overwrite the first timestep’s data
            trackedProfileArrays = copy.deepcopy(profileArraysDictionary)
            continue  # move to next time step — skip accumulation for t=0

        # Add all later times to the initial one
        for key1 in profileArraysDictionary:
            for key2 in profileArraysDictionary[key1]:
                for varName in profileArraysDictionary[key1][key2]:
                    for arrayName in ["profile_array", "profile_array_squares",
                                      "profile_array_left", "profile_array_left_squares",
                                      "profile_array_right", "profile_array_right_squares"]:
                        trackedProfileArrays[key1][key2][varName][arrayName][:, 0:2] += (
                            profileArraysDictionary[key1][key2][varName][arrayName][:, 0:2]
                        )
    return trackedProfileArrays


In [ ]:
if recombine==True:
    for dataName in ["Variables",
                     "Entrainment","PROCESSED_Entrainment",
                     "W_Budgets","QV_Budgets","TH_Budgets"]:
        print(f"Working on {dataName}")
        trackedProfileArrays = RecombineProfiles(ModelData, DataManager_TrackedProfiles)
        TrackedProfiles_DataLoading_CLASS.SaveProfile(ModelData,DataManager_TrackedProfiles, trackedProfileArrays, dataName, t='combined')

In [ ]:
##############################################
#LOADING BACK IN

In [ ]:
# profileArraysDictionary=TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData,DataManager_TrackedProfiles, dataName, t='combined')

In [ ]:
##############################################
#TESTING

In [ ]:
# #testing: comparing above vectorized version to multi-loop version below

# #pure calculation method (slow)
# t=80

# a = trackedArrays['CL']['DEEP']
# ps = a[:,0]
# t1s = a[:,1]
# t2s = a[:,2]+a[:,3]

# profile=np.zeros((ModelData.Nzh,3))
# profile[:,2] = ModelData.zh

# plen = len(ps)
# for count, (p,t1,t2) in enumerate(zip(ps,t1s,t2s)):
#     if count % 100 == 0: print(f"{count}/{plen}")
#     ts = np.arange(t1, t2+1)
    
#     for t_ in ts:
#         if t_ != t:
#             continue
#         Z = GetSpatialData(t)[p]
#         var = MakeDataDictionary(["W"], t=t)["W"][p]
#         profile[Z,0]+=var
#         profile[Z,1]+=1


# #plotting

# def Plot(a,color):
#     b= a[:,0]/a[:,1]
#     # b*=1000
#     plt.plot(b,a[:,2],color=color)
# one = profileArraysDictionary['CL']['DEEP']['W']['profile_array'] #created using MakeTrackedProfiles function
# two = profile.copy()
# Plot(one,color='black')
# Plot(two,color='blue') #plots are the same
# np.all(one[:,0]==two[:,0]) #=True